In [ ]:
ls

### filter for files with fastq extension

In [ ]:
fastq_list = []
for folder, subfolders, files in os.walk('FDAH'):
    for file in files:
        if '.fastq.' in file and not '.md5' in file and not 'trim' in file and not 'sub' in file:
            fastq_list.append(os.path.join(folder,file))
fastq_list

### read in the sample sheet

In [ ]:
samples = pd.read_csv('sample_sheet1.csv', sep='\t', index_col=0)
samples[:5]

### function to retrieve fastqs for specific row of sample sheet

In [ ]:
for file in fastq_list:
    print(file)

In [ ]:
def get_fastq_paths(row):
    for file in fastq_list:
        if row['sample'] in file:
            if 'index' in file:
                row['index'] = file
            elif 'barcode' in file:
                row['barcode'] = file
            elif '_R1' in file:
                row['R1'] = file
            elif '_R2' in file:
                row['R2'] = file
    return row[['name', 'Lane', 'R1', 'R2', 'barcode', 'index']]

### populate sample sheet with fastq paths

In [ ]:
samples = samples.apply(get_fastq_paths, axis=1)
samples[:5]

## get tumor-normal-pairs

In [ ]:
def get_tumor_normal_pairs(samples):
    '''
    turns valid_file_info into tuples of valid tumor normal pairs
    '''

    # extract the tumor-normal suffix
    samples[['sample', 'TN']] = samples['name'].str.extract('(?P<sample>^[^_]+)_(?P<TN>[^_]+)$')
    tumor = 'A'
    normal = 'B'

    def TN_pair(group):
        print(group['name'])
        if tumor in list(group['TN']) and normal in list(group['TN']):
            return f"{group['sample'][0]}_{tumor}-{normal}"
        
    TN_df = pd.DataFrame()
    TN_df['pairs'] = samples.groupby('sample').apply(TN_pair)
    return list(TN_df['pairs'])

In [ ]:
def TN_pair(group):
    print(group['name'])
    if tumor in list(group['TN']) and normal in list(group['TN']):
        return f"{group['sample'][0]}_{tumor}-{normal}"
    
samples.groupby('sample').apply(lambda df: df['name'])

### get sample-type list for multiqc

In [ ]:
def get_fastqc_list():
    fastqc_list = []
    trim_suffix = 'TRIM'
    # create file list from the included_files tuple list
    read1 = 'R1'
    read2 = 'R2'
    trim_suffix = 'TRIM'
    for file_name in samples['name']:
        fastqc_list.append(f"fastQC/{file_name}_{read1}_fastqc.zip")
        fastqc_list.append(f"fastQC/{file_name}_{read2}_fastqc.zip")
        if True: # config['trim']['trimming']:
            fastqc_list.append(f"fastQC/{file_name}_{read1}.{trim_suffix}_fastqc.zip")
            fastqc_list.append(f"fastQC/{file_name}_{read2}.{trim_suffix}_fastqc.zip")
    return fastqc_list

In [ ]:
get_fastqc_list()